In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import dlib
from imutils import face_utils
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from PIL import Image
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch.optim import Adam
from tqdm import tqdm
from glob import glob
import torch.nn.functional as F
import time 
from torchvision import transforms
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import wandb
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision import models
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


In [3]:
class CoordConv2d(nn.Module):

    def __init__(self, device, in_channels, out_channels, kernel_size, padding, stride, input_size):
        super(CoordConv2d, self).__init__()
        self.device = device
        self.cc_xy = self.make_channels(input_size)
        
        
        self.conv = nn.Conv2d(in_channels+2, out_channels,
                              kernel_size=kernel_size, padding=padding, stride=stride)

    def make_channels(self, input_size):
        coord_vals = (2 * torch.arange(input_size) / input_size) - 1

        xchannel = coord_vals.repeat((input_size, 1)).unsqueeze(dim=0)

        ychannel = xchannel.permute(0, 2, 1)
       
        return torch.cat((xchannel.unsqueeze(dim=0), ychannel.unsqueeze(dim=0)), dim=1)

    def forward(self, x):
        n = x.shape[0]
        x = torch.cat((x, self.cc_xy.repeat(
            n, 1, 1, 1).to(self.device)), dim=1)
        
        
        return x
        # print(x.shape)
        # return self.conv(x)


In [4]:
tnsr = torch.randn(1,3,64,64).to("cpu")



In [5]:
conv1 = CoordConv2d(
                "cpu", 3, 3, kernel_size=1, padding=0, stride=1, input_size=64)

conv1(tnsr)


tensor([[[[ 1.6021,  0.0588, -0.4799,  ...,  0.4729, -0.0953,  2.6316],
          [-1.8634,  0.9666, -1.3063,  ...,  0.1334,  0.4021, -0.2607],
          [ 0.5176, -0.8686, -0.8241,  ..., -1.2581,  0.9498, -0.3039],
          ...,
          [ 0.0531, -0.8217, -1.1069,  ..., -0.8360,  0.0057, -0.7618],
          [ 0.1394,  0.9175,  0.7420,  ...,  2.1492, -1.4799,  1.2590],
          [-1.2697,  2.0625,  0.5092,  ..., -0.5785, -1.5654,  0.5615]],

         [[-0.2325, -0.7557,  0.0101,  ...,  0.2498,  0.8970, -0.0193],
          [-0.8498,  1.2583, -2.1959,  ...,  0.7069,  1.7374, -0.1471],
          [-0.6616,  0.5512, -1.2373,  ..., -0.8073, -1.0099,  0.2612],
          ...,
          [-0.0144, -0.0542, -0.8541,  ...,  0.0389,  1.6865, -0.8222],
          [-1.5665,  0.1591,  0.3424,  ...,  1.1934, -1.0794,  0.3710],
          [-0.1498,  0.5400, -0.3635,  ..., -1.0015, -0.1113, -0.2157]],

         [[-0.9845, -0.6154,  1.2181,  ..., -0.9560,  1.6290,  0.3202],
          [-0.3752, -0.2539,  

In [6]:
resnet = models.resnet34(weights=True)
layers = list(resnet.children())[:8]
features1 = nn.Sequential(*layers[:6])



dir(resnet)
        # # for param in resnet.parameters():

        # #     param.requires_grad = False

        # layers = list(resnet.children())[:8]
        # self.
        # self.features2 = nn.Sequential(*layers[6:])

/home/nipun/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_impl',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_make_layer',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_norm_layer',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dic

In [7]:
features1(tnsr).shape

torch.Size([1, 128, 8, 8])

In [8]:
conv2 = nn.Conv2d(3, 3, kernel_size=1, padding=0, stride=1)

In [9]:


class BB_model(nn.Module):
    def __init__(self, device, coordConv=True):
        super().__init__()

        if coordConv:
            self.conv1 = CoordConv2d(
                device, 3, 3, kernel_size=1, padding=0, stride=1, input_size=64)
        else:
            self.conv1 = nn.Conv2d(3, 3, kernel_size=1, padding=0, stride=1)

        resnet = models.resnet34(weights=True)

        # for param in resnet.parameters():

        #     param.requires_grad = False

        layers = list(resnet.children())[:8]
        self.features1 = nn.Sequential(*layers[:6])
        self.features2 = nn.Sequential(*layers[6:])

        # self.fc2
        self.bb = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(128, 2),
        )

    def forward(self, x):

        x = self.conv1(x)

        x = self.features1(x)  # 1, 128, 32, 32

        x = self.features2(x)  # [1, 512, 8, 8]

        x = F.relu(x)

        x = nn.AdaptiveAvgPool2d((1, 1))(x)  # [ 1,512,1,1]

        x = x.view(x.shape[0], -1)  # [1, 512]

        return self.bb(x)


In [10]:
IMAGE_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/MixDataset/images"
trn_df = pd.read_csv("/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/MixDataset/trainAll.csv")
val_df = pd.read_csv("/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/MixDataset/valAll.csv")

RESIZE_AMT = 64
BACTH_SIZE = 512

train_transforms =  A.Compose([
    A.Resize(width=RESIZE_AMT,height=RESIZE_AMT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(p=1)
])

val_transforms =  A.Compose([
    A.Resize(width=RESIZE_AMT,height=RESIZE_AMT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(p=1)
])


In [11]:

class CenterDataset(torch.utils.data.Dataset):
    def __init__(self,df,image_dir=IMAGE_DIR,transforms=None):
        self.image_dir = image_dir
        self.df = df
        self.image_ids = df.Image_Name.unique()
        self.transforms = transforms
        
    def __getitem__(self,ix):
        
        img_id = self.image_ids[ix]
        img_path = os.path.join(self.image_dir,img_id)
        
        img = cv2.imread(img_path)[:,:,::-1]
        
        data = self.df[self.df["Image_Name"]==img_id]
        
        
        x1 = data["X1"].values[0] * RESIZE_AMT
        y1 = data["Y1"].values[0] * RESIZE_AMT
        
        center_loc = torch.Tensor([x1,y1]).float()

        if self.transforms:
            transformed = self.transforms(image=img)
            
            image = transformed["image"]
            
    
        return image,center_loc
    def collate_fn(self,batch):
        return tuple(zip(*batch))
    
    def __len__(self):
        return len(self.image_ids)

In [12]:
train_ds = CenterDataset(trn_df,transforms=train_transforms)
test_ds = CenterDataset(val_df,transforms=val_transforms)

trainLoader = DataLoader(train_ds, batch_size=BACTH_SIZE,
	shuffle=True, num_workers=os.cpu_count(), pin_memory=True,drop_last=True)
testLoader = DataLoader(test_ds, batch_size=BACTH_SIZE,
	num_workers=os.cpu_count(), pin_memory=True,drop_last=True)



def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

In [13]:








def main_training(model,patience, optimizer,scheduler, train_dl, test_dl, epochs,loss_fn):
    
    
    
    idx = 0

    prev_loss = 0
    
    
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    epochTrainLoss = []
    epochValLoss = []
    for i in range(epochs):
        model.train()
        total = 0
        sum_loss = 0

        for x, y_bb in train_dl:
            batch = x.shape[0]
            x = x.cuda().float()

            y_bb = y_bb.cuda()

            
    
            out_bb = model(x)

            loss_bb = loss_fn(out_bb,y_bb).sum(1)
            # loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
            loss_bb = loss_bb.sum()

            loss = loss_bb

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            idx += 1

            total += batch

            sum_loss += loss.item()

        val_loss = val_epochs(model, test_dl,loss_fn)

        
        early_stopping(val_loss,model)
        # if i == 0:
        #     prev_loss = val_loss
        # if val_loss < prev_loss:
        #     prev_loss = val_loss

        #     model_name = f"Regression_model_{str(prev_loss)}.pth"
        #     torch.save(model, model_name)

        train_loss = sum_loss/total
        epochTrainLoss.append(train_loss)
        epochValLoss.append(val_loss)
        
        scheduler.step(train_loss)

        # train_metrics = {"train/epoch": i+1, "train/train_loss": train_loss}

        # val_metrics = {"val/epoch": i+1, "val/val_loss": val_loss}
        # wandb.log({**train_metrics, **val_metrics})

        print(f"Epoch Number {i+1}")
        print("train_loss %.3f " % (train_loss))
        print("Validation Loss %.3f " % (val_loss))
        print("*"*8)
    
    return epochTrainLoss,epochValLoss
        


In [14]:
def val_epochs(model,val_loader,loss_fn):
    
    model.eval()
    total_val_loss = 0
    total = 0
    for x,y_bb in val_loader:
        
    
        x = x.cuda().float()
        y_bb = y_bb.cuda()
        
        out_bb = model(x)
        
        total += x.shape[0]
        with torch.no_grad():
            # loss_bb = F.l1_loss(out_bb,y_bb,reduction='none').sum(1)
            loss_bb = loss_fn(out_bb,y_bb).sum(1)
            loss_bb = loss_bb.sum()
            
            total_val_loss += loss_bb.item()
            
    return total_val_loss/total

In [15]:
n_epoch = 100
    
loss_fn = nn.L1Loss(reduction='none')

model = BB_model(device,coordConv=False).to(device)
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.006)
    
update_optimizer(optimizer, 0.001)
    
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    0.006,
    n_epoch,
    steps_per_epoch=len(train_ds)//BACTH_SIZE
)

    
trainLossModel2,valLossModel2=main_training(model=model,patience=7,optimizer=optimizer,scheduler=scheduler,train_dl=trainLoader,test_dl=testLoader,epochs=n_epoch,loss_fn=loss_fn)
        

/home/nipun/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Validation loss decreased (inf --> 67.712368).  Saving model ...
Epoch Number 1
train_loss 68.779 
Validation Loss 67.712 
********


/home/nipun/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Validation loss decreased (67.712368 --> 60.117210).  Saving model ...
Epoch Number 2
train_loss 66.172 
Validation Loss 60.117 
********
Validation loss decreased (60.117210 --> 53.810474).  Saving model ...
Epoch Number 3
train_loss 59.850 
Validation Loss 53.810 
********
Validation loss decreased (53.810474 --> 44.303318).  Saving model ...
Epoch Number 4
train_loss 49.114 
Validation Loss 44.303 
********
EarlyStopping counter: 1 out of 7
Epoch Number 5
train_loss 28.240 
Validation Loss 58.740 
********
Validation loss decreased (44.303318 --> 19.815474).  Saving model ...
Epoch Number 6
train_loss 11.672 
Validation Loss 19.815 
********
Validation loss decreased (19.815474 --> 11.071307).  Saving model ...
Epoch Number 7
train_loss 9.530 
Validation Loss 11.071 
********
Validation loss decreased (11.071307 --> 7.861758).  Saving model ...
Epoch Number 8
train_loss 9.172 
Validation Loss 7.862 
********
Validation loss decreased (7.861758 --> 7.536084).  Saving model ...
Epoch 

In [ ]:
epoch_x = np.arange(0, n_epoch, 1)


plt.ion()

plt.plot(epoch_x, trainLossModel2, label="Standard", lw=3)
plt.plot(epoch_x, trainLossModel1, label="CoordConv", lw=3)
plt.legend()
plt.title("Supervised Training Regression Error")
plt.xlabel("Epoch")
plt.ylabel("Error")